In [1]:
import numpy as np

In [7]:
class Box:
    def __init__(self, center=[0, 0], radius=[1, 1]):
        if len(center) != len(radius):
            raise ValueError("Center and radius must have the same dimension.")

        self.center = np.array(center, dtype=float)
        self.radius = np.array(radius, dtype=float)

        if np.any(self.radius <= 0):
            raise ValueError("Radius must be positive in every component.")

    def __repr__(self):
        lo = self.center - self.radius
        hi = self.center + self.radius
        return f"Box(center={self.center.tolist()}, radius={self.radius.tolist()})\n" \
               f"Bounds: {[(lo[i], hi[i]) for i in range(len(lo))]}"

    def __eq__(self, other):
        return np.all(self.center == other.center) and np.all(self.radius == other.radius)

    def __add__(self, other):
        if not isinstance(other, (list, np.ndarray, Box)):
            raise TypeError("Can only add list, ndarray, or another Box to a Box.")
        if isinstance(other, Box):
            other = other.center
        if len(other) != len(self.center):
            raise ValueError("Centers must have the same dimension.")
        new_center = self.center + np.array(other, dtype=float)
        return Box(new_center, self.radius)
    
    def __sub__(self, other):
        if not isinstance(other, (list, np.ndarray, Box)):
            raise TypeError("Can only subtract list, ndarray, or another Box to a Box.")
        if isinstance(other, Box):
            other = other.center
        if len(other) != len(self.center):
            raise ValueError("Centers must have the same dimension.")
        new_center = self.center - np.array(other, dtype=float)
        return Box(new_center, self.radius)

    def __mul__(self, other):
        if not isinstance(other, (int, float, list, np.ndarray, Box)):
            raise TypeError("Can only multiply Box by a scalar, a list, or another Box.")
        if isinstance(other, Box):
            if len(other.center) != len(self.center):
                raise ValueError("Boxes must have the same dimension.")
            other = other.radius
        if isinstance(other, (int, float)):
            other = np.array([other] * len(self.center), dtype=float)
        new_radius = self.radius * np.array(other, dtype=float)
        return Box(self.center, new_radius)

    def __truediv__(self, other):
        if not isinstance(other, (int, float, list, np.ndarray, Box)):
            raise TypeError("Can only divide Box by a scalar, a list, or another Box.")
        if isinstance(other, Box):
            if len(other.center) != len(self.center):
                raise ValueError("Boxes must have the same dimension.")
            other = other.radius
        if isinstance(other, (int, float)):
            other = np.array([other] * len(self.center), dtype=float)
        new_radius = self.radius / np.array(other, dtype=float)
        return Box(self.center, new_radius)

    def volume(self):
        return np.prod(2 * self.radius)

    def contains(self, point):
        if len(point) != len(self.center):
            raise ValueError("Point must have the same dimension as the box.")
        return np.all(self.center - self.radius <= np.array(point)) and np.all(np.array(point) < self.center + self.radius)

    def intersect(self, other):
        lo = np.maximum(self.center - self.radius, other.center - other.radius)
        hi = np.minimum(self.center + self.radius, other.center + other.radius)
        if np.any(lo >= hi):
            return None
        return Box((hi + lo) / 2, (hi - lo) / 2)

    def vertices(self):
        n = len(self.center)
        for idx in np.ndindex(*(2,) * n):
            offset = np.array(idx) * 2 - 1  # from (0,1) to (-1,1)
            yield self.center + offset * self.radius

    def subdivide(self, dim):
        new_radius = self.radius.copy()
        new_radius[dim] /= 2
        center1 = self.center.copy()
        center2 = self.center.copy()
        center1[dim] -= new_radius[dim]
        center2[dim] += new_radius[dim]
        return Box(center1, new_radius), Box(center2, new_radius)

    def rescale(self, point):
        """Scale a point from [-1, 1]^N to the box."""
        return self.center + point * self.radius

In [8]:
# Example usage
box1 = Box(center=[0, 0, 0], radius=[1, 1, 1])
print(box1)

point = np.array([0.5, 0.5, 0.5])
print("Contains point:", box1.contains(point))

box2 = Box(center=np.array([1, 1, 1]), radius=np.array([1, 1, 1]))
intersection = box1.intersect(box2)
print("Intersection:", intersection)

print("Volume:", box1.volume())
for vertex in box1.vertices():
    print("Vertex:", vertex)

print(box1 + [1, 1, 1])  # Adding a list
print(box1 - [0.5, 0.5, 0.5])  # Subtracting a list
print(box1 * 2)  # Multiplying by a scalar
print(box1 / 2)  # Dividing by a scalar

Box(center=[0.0, 0.0, 0.0], radius=[1.0, 1.0, 1.0])
Bounds: [(-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0)]
Contains point: True
Intersection: Box(center=[0.5, 0.5, 0.5], radius=[0.5, 0.5, 0.5])
Bounds: [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]
Volume: 8.0
Vertex: [-1. -1. -1.]
Vertex: [-1. -1.  1.]
Vertex: [-1.  1. -1.]
Vertex: [-1.  1.  1.]
Vertex: [ 1. -1. -1.]
Vertex: [ 1. -1.  1.]
Vertex: [ 1.  1. -1.]
Vertex: [1. 1. 1.]
Box(center=[1.0, 1.0, 1.0], radius=[1.0, 1.0, 1.0])
Bounds: [(0.0, 2.0), (0.0, 2.0), (0.0, 2.0)]
Box(center=[-0.5, -0.5, -0.5], radius=[1.0, 1.0, 1.0])
Bounds: [(-1.5, 0.5), (-1.5, 0.5), (-1.5, 0.5)]
Box(center=[0.0, 0.0, 0.0], radius=[2.0, 2.0, 2.0])
Bounds: [(-2.0, 2.0), (-2.0, 2.0), (-2.0, 2.0)]
Box(center=[0.0, 0.0, 0.0], radius=[0.5, 0.5, 0.5])
Bounds: [(-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5)]


In [5]:
class BoxGrid:
    def __init__(self, domain, left, scale, dims):
        self.domain = domain  # Box object
        self.left = np.array(left)
        self.scale = np.array(scale)
        self.dims = np.array(dims)

        if len(domain.center) != len(left):
            raise ValueError("Center and left must have the same dimension.")
        if len(domain.center) != len(scale):
            raise ValueError("Center and scale must have the same dimension.")
        if len(domain.center) != len(dims):
            raise ValueError("Center and dims must have the same dimension.")

    @classmethod
    def from_dims(cls, domain, dims):
        dims = np.array(dims)
        left = domain.center - domain.radius
        scale = dims / (2 * domain.radius)
        return cls(domain, left, scale, dims)

    @classmethod
    def from_domain(cls, domain):
        dims = np.ones_like(domain.center, dtype=int)
        return cls.from_dims(domain, dims)

    def __eq__(self, other):
        return (np.array_equal(self.domain.center, other.domain.center) and
                np.array_equal(self.domain.radius, other.domain.radius) and
                np.array_equal(self.dims, other.dims))

    def __repr__(self):
        return f"{' x '.join(map(str, self.dims))} - element BoxGrid"

    def ndims(self):
        return len(self.dims)

    def size(self):
        return self.dims

    def length(self):
        return np.prod(self.dims)

    def center(self):
        return self.domain.center

    def radius(self):
        return self.domain.radius

    def keys(self):
        # Generating Cartesian indices
        for idx in np.ndindex(*self.dims):
            yield idx

    def subdivide(self, dim):
        new_dims = self.dims.copy()
        new_dims[dim] *= 2
        new_scale = self.scale.copy()
        new_scale[dim] *= 2
        return BoxGrid(self.domain, self.left, new_scale, new_dims)

    def marginal(self, dim):
        center = np.delete(self.domain.center, dim)
        radius = np.delete(self.domain.radius, dim)
        dims = np.delete(self.dims, dim)
        return BoxGrid(Box(center, radius), self.left, self.scale, dims)

    def key_to_box(self, key):
        if not self.check_bounds(key):
            raise IndexError(f"Index {key} is out of bounds for BoxGrid.")
        radius = self.domain.radius / self.dims
        center = self.left + radius + 2 * radius * (np.array(key) - 1)
        return Box(center, radius)

    def check_bounds(self, key):
        return np.all(1 <= np.array(key)) and np.all(np.array(key) <= self.dims)

    def point_to_key(self, point):
        if not self.point_in_domain(point):
            return None
        x = (np.array(point) - self.left) * self.scale
        key = np.floor(x).astype(int) + 1
        if not self.check_bounds(key):
            key = np.clip(key, 1, self.dims)
        return key

    def bounded_point_to_key(self, point):
        # Adjust point to stay within bounds
        center, radius = self.domain.center, self.domain.radius
        small_bound = 1 / (2 * self.scale)
        left = center - radius + small_bound
        right = center + radius - small_bound
        point = np.where(np.isnan(point), np.inf, point)
        point = np.minimum(np.maximum(point, left), right)
        return self.point_to_key(point)

    def point_in_domain(self, point):
        lower_bound = self.domain.center - self.domain.radius
        upper_bound = self.domain.center + self.domain.radius
        return np.all(lower_bound <= np.array(point)) and np.all(np.array(point) < upper_bound)

    def point_to_box(self, point):
        key = self.point_to_key(point)
        if key is None:
            return None
        return self.key_to_box(key)

In [6]:
# Usage example
# Create a Box
domain = Box(center=[0, 0], radius=[1, 1])

# Create a BoxGrid from a domain with dimensions (4, 4)
grid = BoxGrid.from_dims(domain, dims=(4, 4))

# Get the size of the grid
print(grid.size())  # Output: (4, 4)

# Subdivide along the first dimension
new_grid = grid.subdivide(dim=0)
print(new_grid.size())  # Output: (8, 4)

# Map a point to its corresponding box
box = grid.point_to_box([0.5, 0.5])
print(box.center, box.radius)

[4 4]
[8 4]
[0.75 0.75] [0.25 0.25]
